
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [2]:
import numpy as np

# transition matrix
P = np.array([
    [0.3, 0.4, 0.3],
    [0.2, 0.5, 0.3],
    [0.4, 0.3, 0.3]
])

# Downtown: D
# Suburbs: S
# CountrySide: C


In [4]:
# Part 1

# P^2(S,D)
P2 = np.linalg.matrix_power(P, 2)
problem1_p1 = P2[1, 0]
problem1_p1
# problem1_p1 = P2[1, 0] = np.float64(0.28)
# problem1_p1 = 0.28

np.float64(0.28)

In [6]:
# Part 2

# p = p(S->S) x p(S->D) + p(S->C) x p(C->D)

p_S_S_D = P[1, 1] * P[1, 0]
p_S_C_D = P[1, 2] * P[2, 0]

problem1_p2 = p_S_S_D + p_S_C_D
problem1_p2

np.float64(0.22)

In [11]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [12]:
evals,evecs = np.linalg.eig(P.T)
evals,evecs

(array([ 1.00000000e+00, -9.90532031e-17,  1.00000000e-01]),
 array([[-4.93651112e-01, -8.16496581e-01,  7.07106781e-01],
        [-7.02503505e-01,  4.08248290e-01, -7.07106781e-01],
        [-5.12637693e-01,  4.08248290e-01,  4.40033516e-16]]))

In [13]:
evecs[:,0]

array([-0.49365111, -0.70250351, -0.51263769])

In [14]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!

# problem1_stationary = [0.28888889, 0.41111111, 0.3]
stationary = evecs[:,0]/np.sum(evecs[:,0])
stationary

array([0.28888889, 0.41111111, 0.3       ])

In [16]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
# problem1_ET = 3.8444520365145998

# Part 5: Expected hitting time
hitting_times = np.zeros(3)
hitting_times[0] = 0  # H(downtown to downtown) = 0

for _ in range(30):  # Iterate for convergence
    hitting_times[1] = 1 + P[1, 1] * hitting_times[1] + P[1, 2] * hitting_times[2]
    hitting_times[2] = 1 + P[2, 1] * hitting_times[1] + P[2, 2] * hitting_times[2]

expected_hitting_time = hitting_times[1]  # Starting from suburbs
expected_hitting_time

np.float64(3.846132844296536)

---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [18]:

import numpy as np
from scipy import optimize
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param = parameters[0]
    sigma_param = parameters[1]
    n = len(normallySimulatedDataSamples)
    # pdf = (1 / (sigma_param * np.sqrt(2 * np.pi))) * np.exp(-((normallySimulatedDataSamples - mu_param) ** 2) / (2 * sigma_param ** 2))
    log_pdf = np.log(1 / (sigma_param * np.sqrt(2 * np.pi))) - ((normallySimulatedDataSamples - mu_param) ** 2) / (2 * sigma_param ** 2)
    neg_log_likelihood = -np.sum(log_pdf)
    return neg_log_likelihood 

# you should only change XXX below and not anything else
parameter_bounding_box=((-3, 10), (0.1, 3.0)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([2, 1]) # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box)
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 58.63138728236854
        x: [ 9.269e+00  1.708e+00]
      nit: 10
      jac: [ 4.974e-06  2.132e-06]
     nfev: 42
     njev: 14
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [19]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''
    n = len(x)
    lambda_hat = 5 * n / sum(x)
    return lambda_hat

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [40]:

def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    M = 2**32
    a = 1664525
    b = 1013904223
    u = seed

    random_numbers = []
    for _ in range(size):
        u = (a * u + b) % M
        random_numbers.append(u)
    return random_numbers


In [41]:

def problem4_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    generated_values = generator(size=size, seed=seed)
    uniform_samples = [value / period for value in generated_values]
    return uniform_samples


In [42]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    accepted_samples = []
    M = np.pi / 2
    
    for i in range(n_iterations):
        x = uniformGenerator(size=1, seed=seed + i)[0]
        f_x = (np.pi / 2) * np.abs(np.sin(2 * np.pi * x))
        g_x = 1
        r_x = f_x / (M * g_x)
        U = uniformGenerator(size=1, seed=seed + n_iterations + i)[0]
        
        # Accept x if U <= r(x)
        if U <= r_x:
            accepted_samples.append(x)
    
    return accepted_samples

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [43]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem4_uniform(generator=problem4_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [1015568748, 1586005467, 2165703038, 3027450565, 217083232, 1587069247, 3327581586, 2388811721, 70837908, 2745540835]
Uniform sampler [0.23645552527159452, 0.3692706737201661, 0.5042420323006809, 0.7048832636792213, 0.05054362863302231, 0.3695183543022722, 0.7747629624791443, 0.556188570568338, 0.0164932357147336, 0.6392460397910327]
Accept-Reject sampler [0.23645552527159452, 0.23684307769872248, 0.23723063012585044, 0.2376181825529784, 0.23800573498010635, 0.2383932874072343, 0.23878083983436227, 0.23916839226149023, 0.23955594468861818, 0.23994349711574614, 0.2403310495428741, 0.24071860197000206, 0.24110615439713, 0.24149370682425797, 0.24188125925138593, 0.24226881167851388, 0.24265636410564184, 0.2430439165327698, 0.24343146895989776, 0.24381902138702571]


In [44]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.13436424411240122, 0.23796462709189137, 0.23604808973743452, 0.6229016948897019, 0.793340083761663, 0.32383276483316237, 0.2267058593810488, 0.4523795535098186, 0.4745706786885481, 0.2590084917154736, 0.10682853770165568, 0.36152277491407514, 0.18126486333322134, 0.6771258268002703, 0.9056396761745207]
